<a href="https://colab.research.google.com/github/tomonari-masada/course2023-nlp/blob/main/13_in_context_learning_with_LLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# In-Context Learning

## 参考資料
* 「ELYZAが公開した日本語LLM「ELYZA-japanese-Llama-2-7b」についての解説 : (1) 事前学習編」
  * https://zenn.dev/elyza/articles/2fd451c944649d
* "Topic Modeling with Llama 2" （量子化のやり方を参考にしました。）
  * https://towardsdatascience.com/topic-modeling-with-llama-2-85177d01e174

* ランタイムの設定でGPUが使えるようにしておいてください。

## 準備

### 必要なライブラリのインストール

In [ ]:
! pip install transformers datasets accelerate auto-gptq

### 量子化のためにバージョンを指定してbitsandbytesをインストール

In [ ]:
! pip install -U bitsandbytes>=0.39.0

**ここでランタイムを再起動する。**

## データセット
* 今回は、やさしい日本語への書き換えのデータセットを使う。
 * https://huggingface.co/datasets/snow_simplified_japanese_corpus

In [ ]:
import os
import torch
from datasets import load_dataset
import transformers
from transformers import AutoTokenizer, AutoConfig, AutoModelForCausalLM

torch.manual_seed(0)

device = f"cuda:{torch.cuda.current_device()}" if torch.cuda.is_available() else "cpu"
print(device)

In [ ]:
dataset = load_dataset("snow_simplified_japanese_corpus")

In [ ]:
dataset

* 中には、書き換えの前後で文が変わらないものもある。
  * これは、書き換えが不要であるという判断をさせるためのインスタンスと思われる。

* 訓練データとテストデータへランダムに分割する。

In [ ]:
dataset = dataset["train"].train_test_split(test_size=0.2, seed=42)

In [ ]:
len(dataset["train"])

In [ ]:
dataset["train"][0]

In [ ]:
len(dataset["test"])

In [ ]:
dataset["test"][0]

## LLMの準備


* 今回は、elyza/ELYZA-japanese-Llama-2-7b-fast-instructを使う。
  * https://huggingface.co/elyza/ELYZA-japanese-Llama-2-7b-fast-instruct
* だが、Google Colab無料版では、この元のモデルは大きすぎて使えない・・・。
  * 20GB以上のメモリがあるGPUなら使える。
* そこで、bitsandbytesライブラリを利用して量子化する。
  * https://huggingface.co/blog/4bit-transformers-bitsandbytes

### 量子化のコンフィギュレーション

In [ ]:
bnb_config = transformers.BitsAndBytesConfig(
    load_in_4bit=True,  # 4-bit quantization
    bnb_4bit_quant_type='nf4',  # Normalized float 4
    bnb_4bit_use_double_quant=True,  # Second quantization after the first
    bnb_4bit_compute_dtype=torch.bfloat16  # Computation type
)

### 量子化
* 15分ぐらい時間がかかる。
  * 手元のPCなら、3分以内で終わる。

In [ ]:
model_name = "elyza/ELYZA-japanese-Llama-2-7b-fast-instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = transformers.AutoModelForCausalLM.from_pretrained(
    model_name,
    trust_remote_code=True,
    quantization_config=bnb_config,
    device_map="auto",
)
model.eval()

## プロンプト
* LLMがうまく感情分析をしてくれそうなプロンプトを考える。

### プロンプト作成用のヘルパ関数

* Llama 2系言語モデルのプロンプトのテンプレートは、以下の通り。

In [ ]:
"""
<s>[INST] <<SYS>>

{{ System Prompt }}
<</SYS>>
{{ User Prompt }}
 [/INST]
{{ Model Answer }}
"""

### プロンプトを作成するヘルパ関数
* このプロンプトが良い、というつもりではありません・・・。
  * 改良してみてください。

In [ ]:
B_INST, E_INST = "[INST]", "[/INST]答え："
B_SYS, E_SYS = "<<SYS>>\n", "\n<</SYS>>\n\n"
DEFAULT_SYSTEM_PROMPT = "あなたは優秀な日本人のアシスタントです。"

def make_prompt(sample):
  text = f"次のような難しい文があります：\n{sample['original_ja']}\n"
  text += "この文を、小学生でも分かるようなやさしい文に、書き換えてください。\n"
  prompt = "{b_inst} {system}{prompt} {e_inst} ".format(
      b_inst=B_INST,
      system=f"{B_SYS}{DEFAULT_SYSTEM_PROMPT}{E_SYS}",
      prompt=text,
      e_inst=E_INST,
      )
  sample["prompt"] = prompt
  return sample

## やさしい日本語への書き換え

### 生成のためのパイプラインの作成
* https://huggingface.co/docs/transformers/v4.34.0/en/main_classes/pipelines#transformers.pipeline

In [ ]:
generator = transformers.pipeline(
    model=model,
    tokenizer=tokenizer,
    task='text-generation',
    temperature=0.1,
    max_new_tokens=50,
    repetition_penalty=1.1
)

### 生成

In [ ]:
for i in range(10):
  print(f'[{i+1}]' + '-'*80)
  sample = dataset["train"][i]
  original = sample["original_ja"]
  ground_truth = sample["simplified_ja"]
  prompt = make_prompt(sample)["prompt"]
  with torch.no_grad():
    output = generator(prompt, return_full_text=False)
  prediction = output[0]['generated_text']
  print(f"original:\t{original}")
  print(f"prediction:\t{prediction}")
  print(f"ground truth:\t{ground_truth}")

## in-context learning

### 生成のためのパイプラインの作成
* 答えが長くなるので、`max_new_tokens`は多いめの方がいいかもしれません。

In [ ]:
generator = transformers.pipeline(
    model=model,
    tokenizer=tokenizer,
    task='text-generation',
    temperature=0.1,
    max_new_tokens=100,
    repetition_penalty=1.1
)

### プロンプトを作成するヘルパ関数

In [ ]:
B_INST, E_INST = "[INST]", "[/INST]答え："
B_SYS, E_SYS = "<<SYS>>\n", "\n<</SYS>>\n\n"
DEFAULT_SYSTEM_PROMPT = "あなたは優秀な日本人のアシスタントです。"

def make_prompt_with_demonstrations(sample, n_demonstrations=5):
  n_demos = n_demonstrations
  text = "難しい文を、小学生でも分かるようなやさしい文に、書き換えたいです。\n"
  text += "まず、書き換えの例を示します。\n"
  while n_demos > 0:
    index = torch.randint(0, len(dataset["train"]), (1,)).item()
    example = dataset["train"][index]
    if example["original_ja"] == sample["original_ja"]:
      continue
    text += f"次のような難しい文があります：\n「{example['original_ja']}」\n"
    text += "この文は、次のようなやさしい文に、書き換えられます。\n"
    text += f"「{example['simplified_ja']}」\n"
    n_demos -= 1
  text += "では、あなたにお願いします。\n"
  text += f"次のような難しい文があります：\n{sample['original_ja']}\n"
  text += "この文を、小学生でも分かるようなやさしい文に、書き換えてください。\n"
  prompt = "{b_inst} {system}{prompt} {e_inst} ".format(
      b_inst=B_INST,
      system=f"{B_SYS}{DEFAULT_SYSTEM_PROMPT}{E_SYS}",
      prompt=text,
      e_inst=E_INST,
      )
  sample["prompt"] = prompt
  return sample

### 生成

In [ ]:
for i in range(10):
  print(f'[{i+1}]' + '-'*80)
  sample = dataset["train"][i]
  original = sample["original_ja"]
  ground_truth = sample["simplified_ja"]
  sample = make_prompt_with_demonstrations(sample, n_demonstrations=10)
  prompt = sample["prompt"]
  with torch.no_grad():
    output = generator(prompt, return_full_text=False)
  prediction = output[0]['generated_text']
  print(f"original:\t\t{original}")
  print(f"prediction:\t{prediction}")
  print(f"ground truth:\t{ground_truth}")

# 本日の課題
* もっとうまくLLMに書き換えをさせるプロンプトを考えてみよう。